### 0. Drive settings, Imports and Control panel

In [1]:
# Celda para ver cuanta ram tienes

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print(ram_gb)

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!
89.639657472


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#! pip install tensorflow

In [3]:
import glob
import json

import keras
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import random
import skimage.io

import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from keras.layers import concatenate

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torchvision.models as models

import urllib.request
from PIL import Image

In [4]:
#######################################################
# CONTROL PANEL
#######################################################

target_var = "hdi" # either ("hdi" or "gdp")

# Programmed but un-implemented as of yet
train_VGG = True
train_ResNet = True
train_AlexNet = True

### A. Loading and preprocessing data

In [5]:
# Loading X_img, X_popn and Y

features_img = np.load("/content/drive/MyDrive/Thesis/imageset_500_500.npy") 
features_img = features_img.astype('float32')
features_img = features_img/255.0

features_popn = popn_inputs = pd.read_csv("/content/drive/MyDrive/Thesis/siei_processed_v1.csv")[["2019_figure_est"]]

features = np.array(list(zip(features_img, np.array(features_popn))))

if target_var == 'hdi':
    labels = np.load("/content/drive/MyDrive/Thesis/target_HDI.npy") 
    
elif target_var == 'gdp':
    labels = np.load("/content/drive/MyDrive/Thesis/target_logGDP.npy") 

# Split 60% 20% 20%

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.4, random_state=1945, stratify=labels)

# Further split the training set into training and validation sets
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=1945, stratify=y_test)

# Float conversion to allow normalization
#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
#X_val=X_val.astype('float32')

# Normalization 
#X_train=X_train/255.0
#X_test=X_test/255.0
#X_val=X_val/255.0

# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical((y_train-1), num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical((y_val-1), num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical((y_test-1), num_classes=num_classes)

<ipython-input-5-266376720643>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(list(zip(features_img, np.array(features_popn))))


In [6]:
X_train_img = X_train[:,0]
X_val_img = X_val[:,0]
X_test_img = X_test[:,0]

X_train_popn = X_train[:,1]
X_val_popn = X_val[:,1]
X_test_popn = X_test[:,1]

In [7]:
# Giving appropriate shapes to the training, validation and test inputs
# Transform from arrays of arrays (invalid) to unified array

# Image inputs
X_train_img_0 = np.zeros((3075,500,500,3))
for i, element in enumerate(X_train_img):
    X_train_img_0[i] = element

X_val_img_0 = np.zeros((1026,500,500,3))
for i, element in enumerate(X_val_img):
    X_val_img_0[i] = element

X_test_img_0 = np.zeros((1025,500,500,3))
for i, element in enumerate(X_test_img):
    X_test_img_0[i] = element

# Population figure inputs
X_train_popn_0 = np.zeros((3075,))
for i, element in enumerate(X_train_popn):
    X_train_popn_0[i] = element

X_val_popn_0 = np.zeros((1026,))
for i, element in enumerate(X_val_popn):
    X_val_popn_0[i] = element

X_test_popn_0 = np.zeros((1025,))
for i, element in enumerate(X_test_popn):
    X_test_popn_0[i] = element

### B. VGG16

In [9]:
# Import VGG16
vgg16_tl = VGG16(include_top=False, input_shape=(500, 500, 3))

# Freezing VGG16 layers
for layer in vgg16_tl.layers:
    layer.trainable = False

# Entry layer for instance population inputs
integer_input = Input(shape=(1,), name='integer_input')

# Flattening VGG16 output
image_features = layers.Flatten()(vgg16_tl.layers[-1].output)

# Merge VGG16 output with instance population
merged_inputs = Concatenate()([image_features, integer_input])

# Dense layers
dense_1 = layers.Dense(256, activation='relu')(merged_inputs)
dense_2 = layers.Dense(128, activation='relu')(dense_1)
dense_3 = layers.Dense(64, activation='relu')(dense_2)
dense_4 = layers.Dense(32, activation='relu')(dense_3)
dense_f = layers.Dense(16, activation='relu')(dense_4)

# Output layer 
output = layers.Dense(4, activation='softmax')(dense_f)

# Define new model
vgg16_merged_model = Model(inputs=[vgg16_tl.inputs, integer_input], outputs=output)

# Model summary
vgg16_merged_model.summary()

# Compile model
vgg16_merged_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping (Not working as expected)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# Model fit
vgg_train = vgg16_merged_model.fit([X_train_img_0, X_train_popn_0], y_train_onehot, validation_data=([X_val_img_0, X_val_popn_0], y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 500, 500, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 500, 500, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

### C. ResNet50

In [11]:
# Import ResNet50
resnet50_tl = ResNet50(include_top=False, input_shape=(500, 500, 3))

# Freezing ResNet50 layers
for layer in resnet50_tl.layers:
    layer.trainable = False

# Entry layer for instance population inputs
integer_input = Input(shape=(1,), name='integer_input')

# Flattening ResNet50 output
image_features = layers.Flatten()(resnet50_tl.layers[-1].output)

# Merge ResNet50 output with instance population
merged_inputs = Concatenate()([image_features, integer_input])

# Dense layers
#dense_1 = layers.Dense(16384, activation='relu')(flat_1)
#dense_2 = layers.Dense(2048, activation='relu')(flat_1)
#dense_3 = layers.Dense(500, activation='relu')(flat_1)
dense_4 = layers.Dense(256, activation='relu')(merged_inputs)
dense_5 = layers.Dense(64, activation='relu')(dense_4)
dense_f = layers.Dense(16, activation='relu')(dense_5)

# Output layer  
output = layers.Dense(4, activation='softmax')(dense_f)

# Define new model
resnet_merged_model = Model(inputs=[resnet50_tl.inputs, integer_input], outputs=output)

# Model summary
resnet_merged_model.summary()

# Compile model
resnet_merged_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping (Not working as expected)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# Model fit
resnet_train = resnet_merged_model.fit([X_train_img_0, X_train_popn_0], y_train_onehot, validation_data=([X_val_img_0, X_val_popn_0], y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 506, 506, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 250, 250, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

### D. Alexnet

In [ ]:
# Relevant information on image preprocessing for AlexNet that may be worth considering -> https://pytorch.org/hub/pytorch_vision_alexnet/

# Convirtiendo las features ya normalizadas (/255) a pytorch tensor 
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
X_test = torch.tensor(X_test)

# Conviertiendo los targets (onehot) a tensor
y_train_onehot = torch.tensor(y_train_onehot)
y_val_onehot = torch.tensor(y_val_onehot)
y_test_onehot = torch.tensor(y_test_onehot)

# Zip de los objetos anteriores para que el dataloader se comporte de la manera deseada
# Al zip se le añade .permute para que la configuración de las dimensiones de los tensores 
# de entrada coincidan con las que espera el modelo.
Xy_train = list(zip(X_train.permute(0, 3, 1, 2), y_train_onehot))
Xy_val = list(zip(X_val.permute(0, 3, 1, 2), y_val_onehot))
Xy_test = list(zip(X_test.permute(0, 3, 1, 2), y_test_onehot))

# crea los dataloaders para cada conjunto
train_dataloader = DataLoader(Xy_train, batch_size=64, shuffle=True)
val_dataloader = DataLoader(Xy_val, batch_size=64, shuffle=False)
test_dataloader = DataLoader(Xy_test, batch_size=64, shuffle=False)


# Preparar los indices (train / val) -> From RNN task notebook

count = 0
for batch, (x, y) in enumerate(train_dataloader):
    count += (len(x*1))
    
train_indices_A = count


count = 0    
for batch, (x, y) in enumerate(val_dataloader):
    count += (len(x*1))
    
val_indices_A = count

count = 0    
for batch, (x, y) in enumerate(test_dataloader):
    count += (len(x*1))
    
val_test_A = count


<ipython-input-51-74724c07d5c2>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
<ipython-input-51-74724c07d5c2>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val)
<ipython-input-51-74724c07d5c2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
<ipython-input-51-74724c07d5c2>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.

In [ ]:
# Importing pre-trained AlexNet (Excepto esto todo es de GPT4)
AlexNet = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

# Freezing pre-trained layers
for param in AlexNet.features.parameters():
    param.requires_grad = False

# Eliminar la última capa para obtener las características de la imagen
alexnet_features = nn.Sequential(*list(AlexNet.children())[:-1])

# Crear una capa de entrada para el número entero y la capa de concatenación
class ConcatenateInteger(nn.Module):
    def __init__(self):
        super(ConcatenateInteger, self).__init__()
    
    def forward(self, image_features, integer_input):
        integer_input = integer_input.view(-1, 1)
        return torch.cat((image_features, integer_input), dim=1)

concatenate_integer = ConcatenateInteger()

# Crear la red neuronal compuesta por capas densas para la clasificación
classifier = nn.Sequential(
    nn.Linear(256 * 6 * 6 + 1, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 4),
)

# Crear la red completa
class CombinedModel(nn.Module):
    def __init__(self, alexnet_features, concatenate_integer, classifier):
        super(CombinedModel, self).__init__()
        self.alexnet_features = alexnet_features
        self.concatenate_integer = concatenate_integer
        self.classifier = classifier
    
    def forward(self, image, integer_input):
        image_features = self.alexnet_features(image)
        image_features = image_features.view(image_features.size(0), -1)
        combined_features = self.concatenate_integer(image_features, integer_input)
        return self.classifier(combined_features)

model = CombinedModel(alexnet_features, concatenate_integer, classifier)
model = model.double()

# Compilar y entrenar el modelo
# Asegúrate de tener tus datos en las variables X_images, X_integers y y_labels
# X_images debe ser un tensor de PyTorch con las imágenes de tamaño (500, 500, 3)
# X_integers debe ser un tensor de PyTorch con los números enteros
# y_labels debe ser un tensor de PyTorch con las etiquetas de clasificación

# Aquí tienes un ejemplo básico de cómo entrenar el modelo. 
# Puedes ajustar el código de entrenamiento según tus necesidades, 
# incluyendo el uso de conjuntos de datos personalizados, la división de entrenamiento/validación, etc.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    
    output = model(X_train_img_0, X_train_popn_0)
    loss = criterion(output, y_train_onehot)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
X_train_img_0 = torch.tensor(X_train_img_0).permute(0, 3, 1, 2)
X_train_popn_0 = torch.tensor(X_train_popn_0)
y_train_onehot = torch.tensor(y_train_onehot)

In [ ]:
# Congelando los parámetros de la primera mitad del modelo (AlexNet_model.features)
# La otra mitad permanece descongelada (AlexNet_model.classifier)

for param in AlexNet_model.features.parameters():
    param.requires_grad = False

model = AlexNet_model

# verifica si una capa está congelada o descongelada
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f'Capa {name} está congelada')
    else:
        print(f'Capa {name} está descongelada')

# Include correct number of target classes (Originalmente hay mil incluidas)

AlexNet_model.classifier[6] = nn.Linear(1024, 4)

# Modify classifier 4 (En principio esto es opcional)

AlexNet_model.classifier[4] = nn.Linear(4096,1024)

# Resumen del modelo

AlexNet_model.eval()

Capa features.0.weight está congelada
Capa features.0.bias está congelada
Capa features.3.weight está congelada
Capa features.3.bias está congelada
Capa features.6.weight está congelada
Capa features.6.bias está congelada
Capa features.8.weight está congelada
Capa features.8.bias está congelada
Capa features.10.weight está congelada
Capa features.10.bias está congelada
Capa classifier.1.weight está descongelada
Capa classifier.1.bias está descongelada
Capa classifier.4.weight está descongelada
Capa classifier.4.bias está descongelada
Capa classifier.6.weight está descongelada
Capa classifier.6.bias está descongelada


In [ ]:
# Early stopping diseñado por narendra. No implementado por un problema de dependecias relativo a "copy.deepcopy()"

def nn_converged_B(epoch: int, stop_after_epochs: int, validation_loss: torch.Tensor, model: torch.nn.Module) -> bool:
    converged = False
    # (Re)-start the epoch count with the first epoch or any improvement.
    if epoch == 0 or validation_loss < model.best_validation_loss:
        model.best_validation_loss = validation_loss
        model.epochs_since_last_improvement = 0
        model.best_model = copy.deepcopy(model.state_dict())
    else:
        model.epochs_since_last_improvement += 1

    # If no validation improvement over many epochs, stop training.
    if model.epochs_since_last_improvement > stop_after_epochs - 1:
        model.load_state_dict(model.best_model)
        converged = True
    return converged

In [ ]:


device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda"
model = AlexNet_model.to(device)

#Define loss + optimizer
criterion = nn.CrossEntropyLoss() # El mismo que en los otros dos modelos
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def fit(
        train_loader: DataLoader,
        val_loader: DataLoader,
        train_indices: torch.Tensor,
        val_indices: torch.Tensor,
        num_epochs: int = 100,
        print_per_num_epochs: int = 5,
        convergence_num_epochs: int = 15,
    ) -> None:
# Run the training loop
    for epoch in range(num_epochs):
        print(f"epoch {epoch}")
        # Set the model in training mode for gradient evaluation
        model.train()
        # Set current loss value
        train_loss = 0.0
        correct_train = 0
        # Iterate over the DataLoader for training data
        for batch, (inputs_an, targets) in enumerate(train_loader):
            inputs_an = inputs_an.to("cuda")
            targets = targets.to("cuda")

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            out = model(inputs_an)

            # Compute loss
            loss = criterion(out, targets).mean()

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Add to the total of the training loss
            train_loss += loss.item() * len(inputs_an)

        # Once all training batches have been run, get the mean training loss
        train_loss /= train_indices

        # Set the model in evaluation mode so that gradients are not evaluated
        model.eval()

        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for i, (inputs_an, targets) in enumerate(val_loader):
                inputs_an = inputs_an.to("cuda")
                targets = targets.to("cuda")
                
                # Formulate predictions
                preds = model(inputs_an)

                # Compute loss
                loss = criterion(preds, targets).mean()
                val_loss += loss.item() * len(inputs_an)

            # get mean validation loss
            val_loss /= val_indices
    

        if epoch % print_per_num_epochs == 0: 
            print(f"Train Loss after epoch: {epoch}: {train_loss}") 
            print(f"Validation loss after epoch: {epoch}: {val_loss}") 
             

        # EARLY STOPPING con nn_converged - CEGADO
        #if nn_converged_B(epoch, convergence_num_epochs, val_loss, model): 
        #    print(f"Stopping after epoch {epoch} as validation loss was not improving further") 
        #   break 

In [ ]:
fit(train_loader = train_dataloader, val_loader = val_dataloader, train_indices = train_indices_A, val_indices = val_indices_A)

epoch 0
Train Loss after epoch: 0: 0.8536575057351492
Validation loss after epoch: 0: 0.8245534124197783
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
Train Loss after epoch: 5: 0.8222573174984474
Validation loss after epoch: 5: 0.7998132866028457
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
Train Loss after epoch: 10: 0.8050134473215274
Validation loss after epoch: 10: 0.7877727531434034
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
Train Loss after epoch: 15: 0.7646209873610396
Validation loss after epoch: 15: 0.7676318558277907
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
Train Loss after epoch: 20: 0.7608069023271886
Validation loss after epoch: 20: 0.7706054245170794
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
Train Loss after epoch: 25: 0.7100305646997157
Validation loss after epoch: 25: 0.7662129964512459
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
Train Loss after epoch: 30: 0.7016984004121486
Validation loss after epoch: 30: 0.7519390104051686
epoch 31
epoch 32
epoch 33
epoch 34
epoch 